# Introductory tutorial

[![Open in Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/DOV-Vlaanderen/pydov/master?filepath=docs%2Fnotebooks%2FIntroductory_tutorial.ipynb)

pydov provides machine access to the data that can be visualized with the DOV viewer at [this link](https://www.dov.vlaanderen.be/portaal/?module=verkenner)

All the pydov functionalities rely on the existing DOV webservices. An in-depth overview of the available services and endpoints is provided on the [accessing DOV data page](https://pydov.readthedocs.io/en/latest/endpoints.html#endpoints). To retrieve data, pydov uses a combination of the available WFS services and the XML representation of the core DOV data.  

As pydov relies on the XML data returned by the existing DOV webservices, downloading DOV data with pydov is governed by the same [disclaimer](https://www.dov.vlaanderen.be/page/disclaimer) that applies to the other DOV services. Be sure to consult it when using DOV data with pydov!

pydov interfaces a database hosted by the Flemish governement. Therefore, some syntax of the API as well as the descriptions provided by the backend are in Dutch. 

## Use case: gather data for a hydrogeological model

In [2]:
%matplotlib inline
import inspect, sys

In [3]:
# check pydov path
import pydov

### pydov: general info

To get started with pydov you should first determine which information you want to search for. DOV provides a lot of different datasets about soil, subsoil and groundwater of Flanders, some of which can be queried using pydov. See https://pydov.readthedocs.io/en/latest/quickstart.html for the supported datasets.

In this case, to start with a hydrogeological model, we are interested in the hydrostratigraphic interpretation of the borehole data and the groundwater level. These datasets can be found with the following search objects:
- Hydrodstratigrapic interpretation: https://pydov.readthedocs.io/en/latest/reference.html#pydov.search.interpretaties.HydrogeologischeStratigrafieSearch
- Groundwater level:
https://pydov.readthedocs.io/en/latest/reference.html#pydov.search.grondwaterfilter.GrondwaterFilterSearch

Indeed, each of the datasets can be queried using a search object for the specific dataset. While the search objects are different, the workflow is the same for each dataset. Relevant classes can be imported from the pydov.search package, for example if we’d like to query the dataset with hydrogeological interpretations of borehole data:

In [5]:
from pydov.search.interpretaties import HydrogeologischeStratigrafieSearch
hydrosearch = HydrogeologischeStratigrafieSearch()

If you would like some more information, you can query the search object. Sincy pydov interfaces a Flemish database, the descriptions are in Dutch:

In [7]:
hydrosearch.get_description()

'De hydrostratigrafie geeft, op basis van de (gecodeerde) lithologie, een indeling weer naar de al dan niet watervoerende eigenschappen van een bepaald beschreven diepte-interval. Deze interpretatie respecteert de lithostratigrafie van het Tertiair, maar deelt deze anders in. De hiervoor gebruikte standaard is de Hydrogeologische Codering van de Ondergrond van Vlaanderen (HCOV). Deze kan beschouwd worden als de officiele hydrogeologische codering voor het Vlaams Gewest.'

The different fields that are available for objects of the 'Hydrogeologische Stratigrafie' datatype can be requested with the get_fields() method:

In [8]:
fields = hydrosearch.get_fields()
# print available fields
for f in fields.values():
    print(f['name'])

pkey_interpretatie
Type_proef
Proefnummer
pkey_boring
x
y
Z_mTAW
diepte_tot_m
gemeente
Auteurs
Datum
Opdrachten
betrouwbaarheid_interpretatie
Geldig_van
Geldig_tot
diepte_laag_van
diepte_laag_tot
aquifer


You can get more information of a field by requesting it from the fields dictionary:

- name: name of the field
- definition: definition of this field
- cost: currently this is either 1 or 10, depending on the datasource of the field. It is an indication of the expected time it will take to retrieve this field in the output dataframe.
- notnull: whether the field is mandatory or not
- type: datatype of the values of this field

In [15]:
fields['pkey_interpretatie']

{'name': 'pkey_interpretatie',
 'definition': "URL die verwijst naar de gegevens van deze hydrogeologische stratigrafie op de website. Voeg '.xml' toe om een XML voorstelling van deze gegevens te verkrijgen.",
 'type': 'string',
 'notnull': False,
 'query': True,
 'cost': 1}

The fields *pkey_interpretatie* and *pkey_boring* are important identifiers. In this case *pkey_interpretatie* is the unique identifier of this interpretation and is also the **permanent url** where the data can be viewed (~https://www.dov.vlaanderen.be/data/interpretatie/...), or retrieved if you additionally add ... + **.** *xml* 

The *pkey_boring* is the identifier of the borehole from which this interpretation was made. As mentioned before, it is also  the **permanent url** (~https://www.dov.vlaanderen.be/data/boring/...). 

Optionally, if the values of a field have a specific domain the possible values are listed as *values*:

In [14]:
fields['aquifer']['values']

{'0000': 'Onbekend',
 '0100': 'Quartaire aquifersystemen',
 '0110': 'Ophogingen',
 '0120': 'Duinen',
 '0130': 'Polderafzettingen',
 '0131': 'Kleiige polderafzettingen van de kustvlakte',
 '0132': 'Kleiige polderafzettingen van het Meetjesland',
 '0133': 'Kleiige polderafzettingen van Waasland-Antwerpen',
 '0134': 'Zandige kreekruggen',
 '0135': 'Veen-kleiige poelgronden',
 '0140': 'Alluviale deklagen',
 '0150': 'Deklagen',
 '0151': 'Zandige deklagen',
 '0152': 'Zand-lemige deklagen',
 '0153': 'Lemige deklagen',
 '0154': 'Kleiige deklagen',
 '0160': 'Pleistocene afzettingen',
 '0161': 'Pleistoceen van de kustvlakte',
 '0162': 'Pleistoceen van de Vlaamse Vallei',
 '0163': 'Pleistoceen van de riviervalleien',
 '0170': 'Maas- en Rijnafzettingen',
 '0171': 'Afzettingen Hoofdterras',
 '0172': 'Afzettingen Tussenterassen',
 '0173': 'Afzettingen Maasvlakte',
 '0200': 'Kempens Aquifersysteem',
 '0210': 'Kiezeloolietformatie ten noorden van Feldbiss',
 '0211': 'Zandige eenheid boven de Brunssum 

### Query the data with pydov